In [8]:
import requests
import json
import pandas as pd
from clickhouse_connect import get_client
from datetime import datetime, timedelta, date
from dotenv import load_dotenv
import os
import time

load_dotenv()

# Load environment variables
load_dotenv()
IdGuten_Ozon = os.getenv('IdGuten_Ozon')
KeyGuten_Ozon = os.getenv('KeyGuten_Ozon')
clickhouse_password = os.getenv('ClickHouse')

In [13]:


# === Укажите свои данные ===
client_id = IdGuten_Ozon
api_key = KeyGuten_Ozon

# === URL API ===
url = "https://api-seller.ozon.ru/v3/finance/transaction/list" 

# === Параметры запроса ===
headers = {
    "Client-Id": client_id,
    "Api-Key": api_key,
    "Content-Type": "application/json"
}

# Фильтр по дате: например, последние 2 дня
end_date = datetime.utcnow()
start_date = end_date - timedelta(days=2)

body = {
    "filter": {
        "date": {
            "from": start_date.strftime("%Y-%m-%dT%H:%M:%S.000Z"),
            "to": end_date.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        },
        "operation_type": [],  # Все типы операций
        "posting_number": "",  # Все отправления
        "transaction_type": "all"
    },
    "page": 1,
    "page_size": 1000
}

# === Отправка запроса ===
response = requests.post(url, headers=headers, data=json.dumps(body))

# === Обработка ответа ===
if response.status_code == 200:
    print("Успех! Данные получены.")

    # Извлечение списка операций
    operations = response.json().get("result", {}).get("operations", [])

    if not operations:
        print("Нет данных за указанный период.")
    else:
        # Подготовка списка для DataFrame
        rows = []

        for op in operations:
            base = {
                'operation_id': op.get('operation_id'),
                'operation_type': op.get('operation_type'),
                'operation_date': op.get('operation_date'),
                'operation_type_name': op.get('operation_type_name'),
                'delivery_charge': op.get('delivery_charge'),
                'return_delivery_charge': op.get('return_delivery_charge'),
                'accruals_for_sale': op.get('accruals_for_sale'),
                'sale_commission': op.get('sale_commission'),
                'amount': op.get('amount'),
                'type': op.get('type')
            }

            # Добавляем данные из posting
            posting = op.get('posting', {})
            base.update({
                'posting_delivery_schema': posting.get('delivery_schema'),
                'posting_order_date': posting.get('order_date'),
                'posting_posting_number': posting.get('posting_number'),
                'posting_warehouse_id': posting.get('warehouse_id')
            })

            # Если есть items — добавляем первую запись (или можно расширить для нескольких)
            items = op.get('items', [])
            if items:
                item = items[0]
                base.update({
                    'item_name': item.get('name'),
                    'item_sku': item.get('sku')
                })
            else:
                base.update({
                    'item_name': None,
                    'item_sku': None
                })

            # Если есть services — добавляем первую запись
            services = op.get('services', [])
            if services:
                service = services[0]
                base.update({
                    'service_name': service.get('name'),
                    'service_price': service.get('price')
                })
            else:
                base.update({
                    'service_name': None,
                    'service_price': None
                })

            rows.append(base)

        # Создание DataFrame
        df = pd.DataFrame(rows)
        df['project'] = "Ozon-Smart Market-1676213"

else:
    print(f"Ошибка: {response.status_code}")
    print(response.text)
df

C:\Users\User\AppData\Local\Temp\ipykernel_11884\83830791.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_date = datetime.utcnow()


Успех! Данные получены.


,operation_id,operation_type,operation_date,operation_type_name,delivery_charge,return_delivery_charge,accruals_for_sale,sale_commission,amount,type,posting_delivery_schema,posting_order_date,posting_posting_number,posting_warehouse_id,item_name,item_sku,service_name,service_price,project
0,33520513216,MarketplaceRedistributionOfAcquiringOperation,2025-06-08 00:00:00,Оплата эквайринга,0,0,0,0.00,-36.69,other,,2025-06-08 00:53:31,07686282-0097,0,Электрическая зубная щетка Oclean X Pro Digita...,1.140934e+09,MarketplaceRedistributionOfAcquiringOperation,-36.69,Ozon-Smart Market-1676213
1,33520644610,MarketplaceRedistributionOfAcquiringOperation,2025-06-08 00:00:00,Оплата эквайринга,0,0,0,0.00,-14.10,other,,2025-06-08 01:23:41,0156374527-0028,0,Насадки для ежедневной чистки Oclean Easy Clea...,1.997983e+09,MarketplaceRedistributionOfAcquiringOperation,-14.10,Ozon-Smart Market-1676213
2,33520668621,MarketplaceRedistributionOfAcquiringOperation,2025-06-08 00:00:00,Оплата эквайринга,0,0,0,0.00,-89.67,other,,2025-06-08 01:30:10,15723026-0289,0,"Вытяжка кухонная 60 см встраиваемая, 400 м3/ч,...",1.656089e+09,MarketplaceRedistributionOfAcquiringOperation,-89.67,Ozon-Smart Market-1676213
3,33520803764,MarketplaceRedistributionOfAcquiringOperation,2025-06-08 00:00:00,Оплата эквайринга,0,0,0,0.00,-282.89,other,,2025-06-08 02:22:21,04472471-1031,0,Электрическая зубная щетка Oral-B iO 7 White A...,3.098705e+08,MarketplaceRedistributionOfAcquiringOperation,-282.89,Ozon-Smart Market-1676213
4,33520955779,OperationAgentDeliveredToCustomer,2025-06-08 00:00:00,Доставка покупателю,0,0,1522,-258.74,1047.30,orders,FBS,2025-05-25 09:07:12,23265585-0125-1,15936658494000,"Гель для бритья Gillette Series Sensitive, для...",5.711298e+08,MarketplaceServiceItemRedistributionLastMileCo...,-10.36,Ozon-Smart Market-1676213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,33619704336,OperationAgentDeliveredToCustomer,2025-06-10 00:00:00,Доставка покупателю,0,0,2348,-523.60,1824.40,orders,FBO,2025-06-06 06:58:16,85561690-0040-1,18044249781000,Влагомер древесины и строительных материалов C...,6.850679e+08,None,NaN,Ozon-Smart Market-1676213
659,33620074649,OperationAgentDeliveredToCustomer,2025-06-10 00:00:00,Доставка покупателю,0,0,2409,-554.07,1854.93,orders,FBS,2025-06-06 00:42:38,45665854-0288-1,15936658494000,Насадки для стандартной чистки Oclean X Ultra ...,1.712242e+09,None,NaN,Ozon-Smart Market-1676213
660,33620259995,MarketplaceRedistributionOfAcquiringOperation,2025-06-10 00:00:00,Оплата эквайринга,0,0,0,0.00,-85.39,other,,2025-06-10 18:08:07,36471101-0140,0,"Электронож GRAEF EK 508 / 150 Вт, 2 ножа в ком...",1.418778e+09,MarketplaceRedistributionOfAcquiringOperation,-85.39,Ozon-Smart Market-1676213
661,33620539790,OperationAgentDeliveredToCustomer,2025-06-10 00:00:00,Доставка покупателю,0,0,5349,-652.58,4696.42,orders,FBO,2025-06-08 12:10:25,36110487-0802-2,19262731541000,"Вытяжка кухонная 60 см встраиваемая, 400 м3/ч,...",1.656089e+09,None,NaN,Ozon-Smart Market-1676213


In [14]:
df.columns

Index(['operation_id', 'operation_type', 'operation_date',
       'operation_type_name', 'delivery_charge', 'return_delivery_charge',
       'accruals_for_sale', 'sale_commission', 'amount', 'type',
       'posting_delivery_schema', 'posting_order_date',
       'posting_posting_number', 'posting_warehouse_id', 'item_name',
       'item_sku', 'service_name', 'service_price', 'project'],
      dtype='object')

In [15]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [16]:
# Ensure the DataFrame has the correct columns
columns = ['project','operation_id', 'operation_type', 'operation_date',
       'operation_type_name', 'delivery_charge', 'return_delivery_charge',
       'accruals_for_sale', 'sale_commission', 'amount', 'type',
       'posting_delivery_schema', 'posting_order_date',
       'posting_posting_number', 'posting_warehouse_id', 'item_name',
       'item_sku', 'service_name', 'service_price']

# Reorder columns to match the expected order
data = df[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'ozon_finance'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Sample data to insert: [(33520513216, 'MarketplaceRedistributionOfAcquiringOperation', '2025-06-08 00:00:00', 'Оплата эквайринга', 0, 0, 0, 0.0, -36.69, 'other', '', '2025-06-08 00:53:31', '07686282-0097', 0, 'Электрическая зубная щетка Oclean X Pro Digital, Золотой, 84000 движений', 1140934105.0, 'MarketplaceRedistributionOfAcquiringOperation', -36.69, 'Ozon-Smart Market-1676213'), (33520644610, 'MarketplaceRedistributionOfAcquiringOperation', '2025-06-08 00:00:00', 'Оплата эквайринга', 0, 0, 0, 0.0, -14.1, 'other', '', '2025-06-08 01:23:41', '0156374527-0028', 0, 'Насадки для ежедневной чистки Oclean Easy Clean 2 шт синий', 1997982964.0, 'MarketplaceRedistributionOfAcquiringOperation', -14.1, 'Ozon-Smart Market-1676213'), (33520668621, 'MarketplaceRedistributionOfAcquiringOperation', '2025-06-08 00:00:00', 'Оплата эквайринга', 0, 0, 0, 0.0, -89.67, 'other', '', '2025-06-08 01:30:10', '15723026-0289', 0, 'Вытяжка кухонная 60 см встраиваемая, 400 м3/ч, цвет белый, ELIKOR Интегра 60П-40

ProgrammingError: Unrecognized column 'project' in table ozon_finance